This notebook demonstrates how to use OptKeras, a Python package to optimize hyperparameters of Keras Deep Learning Models using Optuna.

Please see the GitHub repository of OptKeras for details:
https://github.com/Minyus/optkeras



##  Set up Google Colab environment

To run in Google Colab, specify a directory in Google Drive. (GPU is recommended.)

To run in an environment other than Google Colab, just skip this code.

In [1]:
""" To use Google Drive with Colab, 
1. set use_google_drive to True, and
2. specify a directory in Google Drive (Modify as in your Google Drive)
(You will need to authorize manually.)
"""
use_google_drive = False
workdir = '/content/drive/My Drive/Colab Notebooks/OptKeras_Example_Output'


import os

try:
    if use_google_drive:
        from google.colab import drive
        drive.mount('/content/drive')
        # Create target directory & all intermediate directories if don't exists
        if not os.path.exists(workdir):
            os.makedirs(workdir)
            print('## Directory: ' , workdir ,  ' was created.') 
        os.chdir(workdir)
        print('## Current working directory: ', os.getcwd())
except:
    print('Run the code without using Google Drive.')
        
try:    
    print('## Check the uptime. (Google Colab reboots every 12 hours)')
    !cat /proc/uptime | awk '{print "Uptime is " $1 /60 /60 " hours (" $1 " sec)"}'
    print('## Check the GPU info')
    !nvidia-smi
    print('## Check the OS') 
    !cat /etc/issue
    print('## Check the Python version') 
    !python --version
    print('## Check the memory')
    !free -h
    print('## Check the disk')
    !df -h
except:
    print('Run the code assuming the environment is not Google Colab.')

## Check the uptime. (Google Colab reboots every 12 hours)
Uptime is 0.0274333 hours (98.76 sec)
## Check the GPU info
Fri May 10 04:11:49 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    17W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+------------------------------------------------

## Install Optuna 0.10.0

In [2]:
!pip install optuna==0.10.0

     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
     |████████████████████████████████| 1.0MB 25.3MB/s 
     |████████████████████████████████| 102kB 35.5MB/s 
     |████████████████████████████████| 51kB 29.1MB/s 
     |████████████████████████████████| 460kB 64.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/d9/a7/05/1ecde925de6085f0c79a1dfdfed50f5a6ecd60c0d55a389890
  Stored in directory: /root/.cache/pip/wheels/de/cf/b3/0eb5c89ea6aa1b49cb41315f9ec139ada8cbffd575bf170d43
  Stored in directory: /root/.cache/pip/wheels/46/23/48/366f0d8b14d436e58ad0aef531b14af8d8beabeb2986704bd5
  Stored in directory: /root/.cache/pip/wheels/92/f0/ac/2ba2972034e98971c3654ece337ac61e546bdeb34ca960dc8c
Successfully built optuna alembic Mako pyperclip


## Install OptKeras 0.0.4

In [3]:
#!pip install optkeras==0.0.4
# Alternatively you can install from the GitHub repository
!pip install git+https://github.com/Minyus/optkeras.git

  Cloning https://github.com/Minyus/optkeras.git to /tmp/pip-req-build-bg94t371
  Running command git clone -q https://github.com/Minyus/optkeras.git /tmp/pip-req-build-bg94t371
  Stored in directory: /tmp/pip-ephem-wheel-cache-a17y4c6s/wheels/ad/c8/9d/67471f9fd7c5fd15961a2e7aa3d15215212d0631ea96df8b5c
Successfully built optkeras


## Import modules

In [4]:
import numpy as np
import pandas as pd

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dense, Conv2D
from keras.layers import MaxPooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax, Nadam
import keras.backend as K

import keras
print('Keras', keras.__version__)

import tensorflow as tf
print('TensorFlow', tf.__version__)

# import Optuna and OptKeras after Keras
import optuna 
print('Optuna', optuna.__version__)

from optkeras.optkeras import OptKeras
import optkeras
print('OptKeras', optkeras.__version__)

# (Optional) Disable messages from Optuna below WARN level.
optuna.logging.set_verbosity(optuna.logging.WARN) 

Using TensorFlow backend.


Keras 2.2.4
TensorFlow 1.13.1
Optuna 0.10.0
OptKeras 0.0.4


## Set up Dataset

In [5]:
dataset_name = 'MNIST'

if dataset_name in ['MNIST']:
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    img_x, img_y = x_train.shape[1], x_train.shape[2]
    x_train = x_train.reshape(-1, img_x, img_y, 1)
    x_test = x_test.reshape(-1, img_x, img_y, 1)   
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    num_classes = 10
    input_shape = (img_x, img_y, 1)

11493376/11490434 [==============================] - 1s 0us/step


In [6]:
print('x_train: ', x_train.shape)
print('y_train', y_train.shape)
print('x_test: ', x_test.shape)
print('y_test', y_test.shape)
print('input_shape: ', input_shape )    

x_train:  (60000, 28, 28, 1)
y_train (60000,)
x_test:  (10000, 28, 28, 1)
y_test (10000,)
input_shape:  (28, 28, 1)


## A simple Keras model

In [7]:
model = Sequential()
model.add(Conv2D(
    filters=32,
    kernel_size=3,
    strides=1,
    activation='relu',
    input_shape=input_shape ))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer=Adam(),
            loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train,
          validation_data=(x_test, y_test), shuffle=True,
          batch_size=512, epochs=2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 7s 114us/step - loss: 0.4450 - acc: 0.8813 - val_loss: 0.2097 - val_acc: 0.9396
Epoch 2/2
60000/60000 [==============================] - 1s 17us/step - loss: 0.1632 - acc: 0.9552 - val_loss: 0.1227 - val_acc: 0.9664


## Optimization of a simple Keras model without pruning

In [8]:
study_name = dataset_name + '_Simple'

""" Step 1. Instantiate OptKeras class
You can specify arguments for Optuna's create_study method and other arguments 
for OptKeras such as enable_pruning. 
"""

ok = OptKeras(study_name=study_name,
              monitor='val_acc',
              direction='maximize')


""" Step 2. Define objective function for Optuna """

def objective(trial):
    
    """ Clear the backend (TensorFlow). See:
    https://www.tensorflow.org/api_docs/python/tf/keras/backend/clear_session
    """
    K.clear_session() 
    
    """ Step 2.1. Define parameters to try using methods of optuna.trial such as 
    suggest_categorical. In this simple demo, try 2*2*2*2 = 16 parameter sets: 
    2 values specified in list for each of 4 parameters 
    (filters, kernel_size, strides, and activation for convolution).
    """    
    model = Sequential()
    model.add(Conv2D(
        filters=trial.suggest_categorical('filters', [32, 64]),
        kernel_size=trial.suggest_categorical('kernel_size', [3, 5]),
        strides=trial.suggest_categorical('strides', [1, 2]),
        activation=trial.suggest_categorical('activation', ['relu', 'linear']),
        input_shape=input_shape ))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(),
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    """ Step 2.2. Specify callbacks(trial) and keras_verbose in fit 
    (or fit_generator) method of Keras model
    """
    model.fit(x_train, y_train, 
              validation_data=(x_test, y_test), shuffle=True,
              batch_size=512, epochs=2,
              callbacks=ok.callbacks(trial),
              verbose=ok.keras_verbose )
    
    """ Step 2.3. Return trial_best_value (or latest_value) """
    return ok.trial_best_value

""" Step 3. Run optimize. 
Set n_trials and/or timeout (in sec) for optimization by Optuna
"""
ok.optimize(objective, timeout = 3*60) # Run for 3 minutes for demo


[2019-05-10 04:12:32.329143] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)
[2019-05-10 04:12:32.617957]  (None) 
[2019-05-10 04:12:34.823713] Trial#: 0, value: 9.734000e-01| Best trial#: 0, value: 9.734000e-01, params: {'filters': 32, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}
[2019-05-10 04:12:36.822416] Trial#: 1, value: 9.191000e-01| Best trial#: 0, value: 9.734000e-01, params: {'filters': 32, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}
[2019-05-10 04:12:38.516447] Trial#: 2, value: 9.329000e-01| Best trial#: 0, value: 9.734000e-01, params: {'filters': 32, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}
[2019-05-10 04:12:40.622794] Trial#: 3, value: 9.233000e-01| Best trial#: 0, value: 9.734000e-01, params: {'filters': 32, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}
[2019-05-10 04:12:42.834727] Trial#: 4, value: 9.699000e-01| Best trial#: 0, value: 9.734000e-01, params: {'filters': 32, 'kernel_size': 5, 'strides': 1

In [9]:
""" Show Results """
print('Best trial number: ', ok.best_trial.number)
print('Best value:', ok.best_trial.value)
print('Best parameters: \n', ok.best_trial.params)

"""
Alternatively, you can access Optuna's study object to, for example, 
get the best parameters as well.
Please note that study.best_trial returns error if optimization trials 
were not completed (e.g. if you interrupt execution) as of Optuna 0.7.0, 
so usage of OptKeras is recommended.
"""
print("Best parameters (retrieved directly from Optuna)", ok.study.best_trial.params)

""" Check the Optuna CSV log file """
pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.optuna_log_file_path, '\n')
display(pd.read_csv(ok.optuna_log_file_path))

""" Check the Keras CSV log file """
pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.keras_log_file_path, '\n')
display(pd.read_csv(ok.keras_log_file_path))

Best trial number:  26
Best value: 0.9808000007629395
Best parameters: 
 {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}
Best parameters (retrieved directly from Optuna) {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}
Data Frame read from MNIST_Simple_Optuna.csv 



,number,state,value,datetime_start,datetime_complete,activation,filters,kernel_size,strides,_Datetime_epoch_begin,_Datetime_epoch_end,_Trial_num,acc,loss,val_acc,val_loss,system_attrs__number
0,0,TrialState.COMPLETE,0.9734,2019-05-10 04:12:32.334166,2019-05-10 04:12:34.752164,relu,32,5,1,2019-05-10 04:12:33.836179,2019-05-10 04:12:34.617735,0,0.960667,0.141006,0.9734,0.097551,0
1,1,TrialState.COMPLETE,0.9191,2019-05-10 04:12:34.752439,2019-05-10 04:12:36.750894,linear,32,5,1,2019-05-10 04:12:35.903388,2019-05-10 04:12:36.602301,1,0.917200,0.295365,0.9191,0.284077,1
2,2,TrialState.COMPLETE,0.9329,2019-05-10 04:12:36.751318,2019-05-10 04:12:38.439588,relu,32,3,2,2019-05-10 04:12:37.760852,2019-05-10 04:12:38.291317,2,0.919717,0.283115,0.9329,0.239063,2
3,3,TrialState.COMPLETE,0.9233,2019-05-10 04:12:38.440136,2019-05-10 04:12:40.549869,linear,32,3,1,2019-05-10 04:12:39.657476,2019-05-10 04:12:40.394920,3,0.918917,0.287663,0.9233,0.269315,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,64,TrialState.COMPLETE,0.9174,2019-05-10 04:15:22.803950,2019-05-10 04:15:24.625037,linear,64,3,2,2019-05-10 04:15:23.902016,2019-05-10 04:15:24.469052,64,0.913367,0.301642,0.9174,0.285233,64
65,65,TrialState.COMPLETE,0.9771,2019-05-10 04:15:24.632874,2019-05-10 04:15:27.765238,relu,64,5,1,2019-05-10 04:15:26.400777,2019-05-10 04:15:27.608736,65,0.971317,0.102817,0.9771,0.077948,65
66,66,TrialState.COMPLETE,0.9778,2019-05-10 04:15:27.772937,2019-05-10 04:15:30.909031,relu,64,5,1,2019-05-10 04:15:29.531844,2019-05-10 04:15:30.750710,66,0.970183,0.106406,0.9778,0.074906,66
67,67,TrialState.COMPLETE,0.9781,2019-05-10 04:15:30.917168,2019-05-10 04:15:34.120557,relu,64,5,1,2019-05-10 04:15:32.712900,2019-05-10 04:15:33.954044,67,0.970683,0.104216,0.9781,0.073627,67


Data Frame read from MNIST_Simple_Keras.csv 



,epoch,_Datetime_epoch_begin,_Datetime_epoch_end,_Trial_num,acc,loss,val_acc,val_loss
0,0,2019-05-10 04:12:32.812828,2019-05-10 04:12:33.785225,0,0.888283,0.414493,0.9481,0.183785
1,1,2019-05-10 04:12:33.836179,2019-05-10 04:12:34.617735,0,0.960667,0.141006,0.9734,0.097551
2,0,2019-05-10 04:12:35.023023,2019-05-10 04:12:35.859126,1,0.879000,0.434390,0.9170,0.292161
3,1,2019-05-10 04:12:35.903388,2019-05-10 04:12:36.602301,1,0.917200,0.295365,0.9191,0.284077
...,...,...,...,...,...,...,...,...
132,0,2019-05-10 04:15:28.084092,2019-05-10 04:15:29.480333,66,0.902067,0.355802,0.9623,0.136872
133,1,2019-05-10 04:15:29.531844,2019-05-10 04:15:30.750710,66,0.970183,0.106406,0.9778,0.074906
134,0,2019-05-10 04:15:31.242569,2019-05-10 04:15:32.654711,67,0.903500,0.352013,0.9632,0.132535
135,1,2019-05-10 04:15:32.712900,2019-05-10 04:15:33.954044,67,0.970683,0.104216,0.9781,0.073627


## Optimization of a Keras model using more Optuna's features such as pruning

In [10]:
study_name = dataset_name + '_Optimized'

ok = OptKeras( 
    # parameters for optuna.create_study
    storage='sqlite:///' + study_name + '_Optuna.db', 
    sampler=optuna.samplers.TPESampler(
        consider_prior=True, prior_weight=1.0, 
        consider_magic_clip=True, consider_endpoints=False, 
        n_startup_trials=10, n_ei_candidates=24, 
        seed=None), 
    pruner=optuna.pruners.SuccessiveHalvingPruner(
        min_resource=1, reduction_factor=4, min_early_stopping_rate=0), 
    study_name=study_name,
    load_if_exists=True,
    # parameters for OptKeras
    monitor='val_acc',
    direction='maximize',
    enable_pruning=True, 
    models_to_keep=1, # Either 1, 0, or -1 (save all models) 
    verbose=1,
    )

def objective(trial): 
    epochs = 10
    
    K.clear_session()   
    model = Sequential()
    
    if trial.suggest_int('Conv', 0, 1):  
        # 1 Convolution layer
        i = 1
        model.add(Conv2D(
            filters=int(trial.suggest_discrete_uniform(
                'Conv_{}_num_filters'.format(i), 32, 64, 32)), 
            kernel_size=tuple([trial.suggest_int(
                'Conv_{}_kernel_size'.format(i), 2, 3)] * 2),
            activation='relu',
            input_shape=input_shape))
        model.add(MaxPooling2D(pool_size=tuple([trial.suggest_int(
                'Conv_{}_max_pooling_size'.format(i), 2, 3)] * 2)))
        model.add(Dropout(trial.suggest_discrete_uniform(
                'Conv_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        model.add(Flatten())        
    else:
        model.add(Flatten(input_shape=input_shape))
    # 2 Fully connected layers
    for i in np.arange(2) + 1:
        model.add(Dense(int(trial.suggest_discrete_uniform(
            'FC_{}_num_hidden_units'.format(i), 256, 512, 256))))
        if trial.suggest_int('FC_{}_batch_normalization'.format(i), 0, 1):
            model.add(BatchNormalization())
        model.add(Activation(trial.suggest_categorical(
            'FC_{}_acivation'.format(i), ['relu'])))
        model.add(Dropout(
            trial.suggest_discrete_uniform(
                'FC_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        
    # Output layer    
    model.add(Dense(num_classes, activation='softmax'))
    
    optimizer_dict = { \
    #'Adagrad': Adagrad(),
    'Adam': Adam() }
    
    model.compile(optimizer = optimizer_dict[
        trial.suggest_categorical('Optimizer', list(optimizer_dict.keys()))],
          loss='sparse_categorical_crossentropy', metrics=['accuracy'])    
    
    if ok.verbose >= 2: model.summary()
    
    batch_size = trial.suggest_int('Batch_size', 256, 256) 
    data_augmentation = trial.suggest_int('Data_augmentation', 0, 1)
    
    if not data_augmentation:
        # [Required] Specify callbacks(trial) in fit method
        model.fit(x_train, y_train, batch_size=batch_size,
                  epochs=epochs, validation_data=(x_test, y_test),
                  shuffle=True,
                  callbacks=ok.callbacks(trial),
                  verbose=ok.keras_verbose )
    
    if data_augmentation:
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            width_shift_range=[-1, 0, +1], # 1 pixel
            height_shift_range=[-1, 0, +1], # 1 pixel
            zoom_range=[0.95,1.05],  # set range for random zoom
            horizontal_flip=False,  # disable horizontal flip
            vertical_flip=False )  # disable vertical flip
        datagen.fit(x_train)
        # [Required] Specify callbacks(trial) in fit_generator method
        model.fit_generator(datagen.flow(x_train, y_train, 
                                         batch_size=batch_size),
                            epochs=epochs, validation_data=(x_test, y_test),
                            steps_per_epoch=len(x_train) // batch_size,
                            callbacks=ok.callbacks(trial),
                            verbose=ok.keras_verbose )
    
    # [Required] return trial_best_value (recommended) or latest_value
    return ok.trial_best_value

# Set n_trials and/or timeout (in sec) for optimization by Optuna
ok.optimize(objective, timeout=3*60) # Run for 3 minutes for demo


[2019-05-10 04:15:34.581950] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[2019-05-10 04:15:35.127617]  (None) 
[2019-05-10 04:17:20.868803] Trial#: 0, value: 9.848000e-01| Best trial#: 0, value: 9.848000e-01, params: {'Batch_size': 256, 'Conv': 0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 0, 'FC_1_dropout_rate': 0.0, 'FC_1_num_hidden_units': 256.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 0, 'FC_2_dropout_rate': 0.5, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}
[2019-05-10 04:17:51.646969] Trial#: 3, value: 9.886000e-01| Best trial#: 3, value: 9.886000e-01, params: {'Batch_size': 256, 'Conv': 1, 'Conv_1_dropout_rate': 0.25, 'Conv_1_kernel_size': 3, 'Conv_1_max_pooling_size': 2, 'Conv_1_num_filters': 32.0, 'Data_augmentation': 0, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 0,

## Randomized Grid Search of a simple Keras model

In [11]:
study_name = dataset_name + '_GridSearch'

""" To run randomized grid search, set random_grid_search_mode True """
ok = OptKeras(study_name=study_name, random_grid_search_mode=True)


def objective(trial):
    K.clear_session()

    model = Sequential()
    model.add(Conv2D(
        filters=trial.suggest_categorical('filters', [32, 64]),
        kernel_size=trial.suggest_categorical('kernel_size', [3, 5]),
        strides=trial.suggest_categorical('strides', [1]),
        activation=trial.suggest_categorical('activation', ['relu', 'linear']),
        input_shape=input_shape))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.fit(x_train, y_train,
              validation_data=(x_test, y_test), shuffle=True,
              batch_size=512, epochs=2,
              callbacks=ok.callbacks(trial),
              verbose=ok.keras_verbose)

    return ok.trial_best_value

""" Set the number of parameter sets as n_trials for complete grid search """
ok.random_grid_search(objective, n_trials=2*2*2)  # 2*2*2 = 8 param sets


[2019-05-10 04:19:42.211378] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)
[2019-05-10 04:19:42.303273]  (None) 
[2019-05-10 04:19:44.613135] Completed:  12% (    1 /     8)
[2019-05-10 04:19:44.691397] Trial#: 0, value: 2.781258e-01| Best trial#: 0, value: 2.781258e-01, params: {'filters': 32, 'kernel_size': 3, 'strides': 1, 'activation': 'linear'}
[2019-05-10 04:19:46.925172] Completed:  25% (    2 /     8)
[2019-05-10 04:19:47.003790] Trial#: 1, value: 9.391315e-02| Best trial#: 1, value: 9.391315e-02, params: {'filters': 32, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}
[2019-05-10 04:19:50.439585] Completed:  38% (    3 /     8)
[2019-05-10 04:19:50.518369] Trial#: 2, value: 9.512007e-02| Best trial#: 1, value: 9.391315e-02, params: {'filters': 32, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}
[2019-05-10 04:19:53.616758] Completed:  50% (    4 /     8)
[2019-05-10 04:19:53.696223] Trial#: 3, value: 7.454754e-02| Best trial#: 3, value

In [12]:
""" ## The end of code. """

' ## The end of code. '

Please feel free to post questions or feedback [here](
https://github.com/Minyus/optkeras/issues
)
